# **Modelling Loan Defaults**

## Objectives

* Fit and evaluate a classification model to predict if a borrower will default or not

## Inputs

* outputs/datasets/collection/LoanDefaultData.csv
* Instructions on which variables to use for data cleaning and feature engineering. They are found in each respective notebook.

## Outputs

* Train set (features and target)
* Test set (features and target)
* Data cleaning and Feature Engineering pipeline
* Modeling pipeline
* Feature importance plot


>Note: The approach in this notebook is based on the Churnometer Project from Code Institute

---

# Imports

In [ ]:
# System & OS
import os
import warnings  # for ignoring warnings

# Ignore FutureWarnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Data Handling
import pandas as pd
import numpy as np

# Visualization
# Inline plotting for VS Code / Jupyter
%matplotlib inline  
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")  # consistent style

# Machine Learning Utilities
from sklearn.pipeline import Pipeline
from sklearn.model_selection import (train_test_split, 
                                     GridSearchCV, 
                                     RandomizedSearchCV)
from sklearn.preprocessing import StandardScaler  # feature scaling
from sklearn.feature_selection import SelectFromModel  # feature selection
from imblearn.over_sampling import SMOTE  # handle imbalanced datasets

# Feature Engineering (Feature-Engine)
from feature_engine.encoding import OrdinalEncoder
from feature_engine.imputation import MeanMedianImputer
from feature_engine.outliers import Winsorizer
from feature_engine.selection import SmartCorrelatedSelection
from feature_engine import transformation as vt  # log/power transformations

# Classification Algorithms
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (RandomForestClassifier, ExtraTreesClassifier, 
                              GradientBoostingClassifier, AdaBoostClassifier)
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# Performance Metrics
from sklearn.metrics import (classification_report, confusion_matrix, 
                             recall_score, f1_score, make_scorer)

# Joblib for saving/loading models
import joblib

---

# Change working directory

We need to change the working directory from its current folder, where the notebook is stored, to its parent folder
* First we access the current directory with os.getcwd()

In [ ]:
current_dir = os.getcwd()
current_dir

* Then we want to make the parent of the current directory the new current directory
    * os.path.dirname() gets the parent directory
    * os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
current_dir = os.getcwd()
print(f"You set a new current directory: {current_dir}")

---

# Step 1: Load Data

Load the data from the “collection” folder, since we want to design the pipeline to be able to handle the cleaning and engineering by itself.

In [ ]:
df = pd.read_csv("outputs/datasets/collection/LoanDefaultData.csv")

print(df.shape)
df.head(3)

### Drop duplicates

In [ ]:
df = df.drop_duplicates()
print(df.shape)

### Set target variable

In [ ]:
target_var = "loan_status"

---

# Step 2: ML Pipeline with all features

1. First create ML Pipelines:
    * the first three for cleaning and feature engineering 
    * the fourth for modelling using all available data to predict a borrowers's default. We won’t use this pipeline until a later stage in our notebook, but it’s good practice to create all your pipelines at the start of your notebook

2. Next, we split the data into train and test sets

3. After that, we can handle the target imbalance, by applying the cleaning and feature engineering pipelines to the data and then resampling the train set target

4. Once the data is ready, we fit the modelling pipeline we created using multiple algorithms while using their default hyperparameters, so we can find the algorithms that show the best fit for our data

5. Next, for these best given algorithm, we do an extensive optimization search so we can find the best hyperparameters for this algorithm and can identify the best model

6. After that we’ll need to assess the best model's most relevant features from the second pipeline

7. And finally, we evaluate the pipeline using the best model on the train and test sets to check if it meets the requirements we set in our business case

## ML pipeline for Data Cleaning and Feature Engineering

First we apply what we know from the previous feature engineering step.

In [ ]:
def PipelineDataClAndFeatureEngPart1():
    pipeline_base = Pipeline([
        ("MedianImputer", MeanMedianImputer(
            imputation_method='median',
            variables=['person_emp_length', 'loan_int_rate']
        )),
        ("Winsorizer", Winsorizer(
            capping_method='iqr',
            fold=5,
            tail='right',
            variables=[
                'person_age', 'person_income', 'person_emp_length',
                'loan_amnt', 'loan_int_rate', 'loan_percent_income',
                'cb_person_cred_hist_length'
            ]
        )),
        ('log_transform', vt.LogTransformer(
            variables=[
                'person_age', 'person_income', 'cb_person_cred_hist_length'
            ]
        )),
        ('power_transform', vt.PowerTransformer(
            variables=[
                'person_emp_length', 'loan_amnt', 'loan_percent_income'
            ]
        )),
        ("OrdinalEncoderArbitrary", OrdinalEncoder(
            encoding_method='arbitrary',
            variables=['person_home_ownership', 'loan_intent', 
                    'cb_person_default_on_file'],
            unseen='encode'
        ))
    ])

    return pipeline_base

PipelineDataClAndFeatureEngPart1()

The categorical encoder for `loan_grade` has to be directly applied on the `loan_grade` column and therefore has to be applied seperately from the other feature engineering steps.

In [ ]:
def PipelineDataClAndFeatureEngPart2():
    pipeline_base = Pipeline([
        ("OrdinalEncoderOrdered", OrdinalEncoder(
            encoding_method='ordered', 
            variables="loan_grade", 
            unseen='encode'))
    ])

    return pipeline_base

PipelineDataClAndFeatureEngPart2()

After all the previous transformations we still need to apply SmartCorrelatedSelection

In [ ]:

def PipelineDataClAndFeatureEngPart3():
    pipeline_base = Pipeline([
        ("SmartCorrelatedSelection", 
         SmartCorrelatedSelection(variables=None, 
                                  method="spearman", 
                                  threshold=0.6, 
                                  selection_method="variance"))
    ])

    return pipeline_base

PipelineDataClAndFeatureEngPart3()

## ML Pipeline for Modelling and Hyperparameter Optimisation

In [ ]:
%pip install catboost

In [ ]:
def PipelineClf(model):
    pipeline_base = Pipeline([
        ("scaler", StandardScaler()),
        ("feat_selection", SelectFromModel(model)),
        ("model", model),
    ])

    return pipeline_base

## Split Train and Test Set

At this point, we use all 11 features.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop([target_var], axis=1),
    df[target_var],
    test_size=0.2,
    random_state=0,
)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
X_train.head(3)

## Handle Target Imbalance

From our previous analysis we know we have to handle target imbalance to improve performance.

### Preparation: Data Cleaning and Feature Engineering

First apply PipelineDataClAndFeatureEngPart1(), PipelineDataClAndFeatureEngPart2() and PipelineDataClAndFeatureEngPart3(), because for resampling, the data can’t contain missing values
or categorical values. 

In [ ]:
pipeline_data_cl_feat_eng_part1 = PipelineDataClAndFeatureEngPart1()
X_train = pipeline_data_cl_feat_eng_part1.fit_transform(X_train)
X_test = pipeline_data_cl_feat_eng_part1.transform(X_test)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
pipeline_data_cl_feat_eng_part2 = PipelineDataClAndFeatureEngPart2()
X_train["loan_grade"] = (
    pipeline_data_cl_feat_eng_part2
    .fit_transform(X_train[["loan_grade"]],y_train)
    )
X_test["loan_grade"] = (
    pipeline_data_cl_feat_eng_part2
    .transform(X_test[["loan_grade"]])
    )
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
pipeline_data_cl_feat_eng_part3 = PipelineDataClAndFeatureEngPart3()
X_train = pipeline_data_cl_feat_eng_part3.fit_transform(X_train)
X_test = pipeline_data_cl_feat_eng_part3.transform(X_test)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

After applying the data cleaning and feature engineering pipeline, the number of features decreased from 11 to 8, because of ``SmartCorrelatedSelection``.

#### Apply SMOTE

Check current Train Set Target distribution to confirm the target is currently imbalanced.

In [ ]:
sns.set_style("whitegrid")
y_train.value_counts().plot(kind='bar', title='Train Set Target Distribution')
plt.show()

Use SMOTE (Synthetic Minority Oversampling Technique) to balance the Train Set target.

In [ ]:
oversample = SMOTE(sampling_strategy='minority', random_state=0)
X_train, y_train = oversample.fit_resample(X_train, y_train)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

Check Train Set Target distribution after resampling and confirm that the target is now balanced.

In [ ]:
y_train.value_counts().plot(kind='bar', title='Train Set Target Distribution')
plt.show()

## Grid Search CV - Sklearn

This step is divided into 2 subsections:
* First, we will fit a set of pipelines for each algorithm using the default hyperparameters,
to find the algorithm that most suits the data
* Then, we will do an extensive hyperparameter search using this algorithm

This combined approach is faster than doing one extensive search on all algorithms at once.

### Custom Class for Hyperparameter Optimisation

We use the custom class introduced by Code Institute in the "Churnometer" walkthrough project to assist with hyperparameter optimization. We customized the class to support both RandomizedSearchCV and GridSearchCV. The RandomizedSearchCV option was added to reduce computation time during the extensive hyperparameter search.

The class automates the hyperparameter tuning process across multiple models, each with its own parameter grid.

In the fit() method, each model is wrapped in a custom PipelineClf() and passed to GridSearchCV/RandomizedSearchCV, which performs an exhaustive search over the defined hyperparameter combinations using cross-validation. The fitted grid search objects are stored for later analysis.

The score_summary() method then compiles the cross-validation results into a unified DataFrame, summarizing each model’s minimum, maximum, mean, and standard deviation of scores, enabling quick and consistent comparison across all tested algorithms.

In [ ]:
# Customized from Churnometer Walkthrough Project
class HyperparameterOptimizationSearch:

    def __init__(self, models, params):
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}

    def fit(self, X, y, cv, n_jobs, n_iter=30, randomized=False, 
            verbose=1, scoring=None, refit=False):
        
        for key in self.keys:

            model = PipelineClf(self.models[key])
            params = self.params[key]
            if randomized == True:
                print(f"\nRunning RandomizedSearchCV for {key} \n")
                gs = RandomizedSearchCV(model, params, cv=cv, n_jobs=n_jobs,
                                verbose=verbose, scoring=scoring, n_iter=n_iter)
            else: 
                print(f"\nRunning GridSearchCV for {key} \n")
                gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs, 
                                  verbose=verbose, scoring=scoring, )
            gs.fit(X, y)
            self.grid_searches[key] = gs

    def score_summary(self, sort_by='mean_score'):
        def row(key, scores, params):
            d = {
                'estimator': key,
                'min_score': min(scores),
                'max_score': max(scores),
                'mean_score': np.mean(scores),
                'std_score': np.std(scores),
            }
            return pd.Series({**params, **d})

        rows = []
        for k in self.grid_searches:
            params = self.grid_searches[k].cv_results_['params']
            scores = []
            for i in range(self.grid_searches[k].cv):
                key = "split{}_test_score".format(i)
                r = self.grid_searches[k].cv_results_[key]
                scores.append(r.reshape(len(params), 1))

            all_scores = np.hstack(scores)
            for p, s in zip(params, all_scores):
                rows.append((row(k, s, p)))

        df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)
        columns = ['estimator', 'min_score',
                   'mean_score', 'max_score', 'std_score']
        columns = columns + [c for c in df.columns if c not in columns]
        return df[columns], self.grid_searches

### Custom Performance Metric

To evaluate model performance we use a hybrid score combining recall and F1 because the target is imbalanced and catching defaults is critical. This ensures the model prioritizes identifying true defaults (high recall) while still considering overall predictive quality (F1), guiding hyperparameter tuning toward a practical, business-relevant performance.

In [ ]:
def hybrid_score(y_true, y_pred):
    recall = recall_score(y_true, y_pred, pos_label=1)
    f1_macro = f1_score(y_true, y_pred, average='macro')
    return 0.7 * recall + 0.3 * f1_macro

### Use standard hyperparameters to find most suitable algorithms

#### Define algorithms for Quick Search

In [ ]:
models_quick_search = {
    # Linear models
    "LogisticRegression": LogisticRegression(random_state=0),
    "RidgeClassifier": RidgeClassifier(random_state=0),

    #Tree-based models
    "DecisionTreeClassifier": DecisionTreeClassifier(random_state=0),
    "RandomForestClassifier": RandomForestClassifier(random_state=0),
    "ExtraTreesClassifier": ExtraTreesClassifier(random_state=0),

    # Boosting methods 
    "XGBClassifier": XGBClassifier(random_state=0),
    "CatBoostClassifier": CatBoostClassifier(random_state=0, verbose=0),
    "AdaBoostClassifier": AdaBoostClassifier(random_state=0),
    "GradientBoostingClassifier": GradientBoostingClassifier(random_state=0)
}

# Set empty dictionaries to indicated that we use the default hyperparameters
params_quick_search = {
    "LogisticRegression": {},
    "RidgeClassifier": {},
    "XGBClassifier": {},
    "CatBoostClassifier": {},
    "DecisionTreeClassifier": {},
    "RandomForestClassifier": {},
    "GradientBoostingClassifier": {},
    "ExtraTreesClassifier": {},
    "AdaBoostClassifier": {},
}

For our binary classification task, we select a diverse set of models that cover different algorithmic families and learning principles. This allows us to compare performance across linear, tree-based, and ensemble approaches:

* Linear Models:  
Serve as strong and interpretable baselines. They model linear relationships between features and the target and provide useful insights into feature importance.

* Tree-Based Models:  
Capture nonlinear relationships and feature interactions without requiring scaling or transformation. Random Forest and Extra Trees improve generalization by aggregating multiple decision trees.

* Boosting Methods:  
Build powerful ensembles by sequentially improving weak learners. These algorithms often achieve high accuracy on structured data and handle complex feature relationships effectively.

This model mix ensures a comprehensive benchmark, from simple and interpretable models to complex and high-performing ensemble methods, helping us identify the most suitable approach for our dataset.

#### Quick GridSearch CV

Now, we perform a quick GridSearchCV to evaluate the model performance of the different algorithms using the default hyperparameters.

We use 5-fold cross-validation to obtain a robust estimate of model performance and reduce the impact of random variation in the training data. It is a rule of thumb used in data science as a starting point.

In [ ]:
search = HyperparameterOptimizationSearch(models=models_quick_search, 
                                          params=params_quick_search)
search.fit(X_train, y_train,
           scoring = make_scorer(hybrid_score),
           n_jobs=-1, cv=5)

#### Check results

In [ ]:
grid_search_summary, grid_search_pipelines = (search
                                              .score_summary(
                                                  sort_by='mean_score'))
grid_search_summary 

* Among the tested algorithms, the best-performing model is the ``RandomForestClassifier``, achieving a mean Recall score of 0.847. The second-best model is the ``DecisionTreeClassifier``, with a Recall score of 0.832.

In a professional setting, we could continue tuning hyperparameters for both models, as their scores are both high and offer strong potential. However, to reduce computing time while still meeting the project requirements defined by Code Institute, we will focus only on the ``RandomForestClassifier`` for the next steps.

### Do an extensive search on the RandomForestClassifier to find the best hyperparameter configuration

Now we will perform an extensive grid search using the RandomForestClassifier to optimize its hyperparameters for the best possible performance.

#### Define algorithm and hyperparameters for Extensive Search

In [ ]:
models_search = {
    "RandomForestClassifier":RandomForestClassifier(random_state=0)
}

# Documentation to help on hyperparameter list: 
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
# https://www.geeksforgeeks.org/machine-learning/hyperparameters-of-random-forest-classifier/
   
params_search = {
    "RandomForestClassifier": {
        'model__n_estimators': [500, 800, 1000],  
        'model__criterion': ["gini", "log_loss", "entropy"],      
        'model__max_depth': [6, 8, 10],           
        'model__min_samples_split': [2, 5, 10],         
        'model__min_samples_leaf': [1, 3, 5],           
        'model__max_features': ["sqrt", "log2", 0.5],   
        'model__class_weight': ['balanced', None]       
    }
}

#### Extensive GridSearch CV

In [ ]:
search = HyperparameterOptimizationSearch(models=models_search, 
                                          params=params_search)

# use RandomizedSearchCV to reduce computing time
search.fit(X_train, y_train,
           scoring = make_scorer(hybrid_score),
           n_jobs=-1, cv=5, randomized=True, n_iter=50) 

#### Check results

In [ ]:
grid_search_summary, grid_search_pipelines = (search
                                              .score_summary(
                                                  sort_by='mean_score'))
grid_search_summary.head()

Get best model name programmatically

In [ ]:
best_model = grid_search_summary.iloc[0,0]
best_model

The best RandomForestClassifier model has a mean score of 0.8 and the following hyperparameters:

In [ ]:
best_parameters = grid_search_pipelines[best_model].best_params_
best_parameters

Extract the best classification pipeline from the GridSearchCV object, which includes the preprocessing steps and the model fitted with the optimal hyperparameters found during the grid search.

In [ ]:
pipeline_clf = grid_search_pipelines[best_model].best_estimator_
pipeline_clf

## Assess feature importance

Our classification pipeline consists of three steps:

1. Feature scaling – standardizes the numerical variables.
2. Feature selection – selects the most relevant features for the model.
3. Modelling – fits the RandomForestClassifier on the selected features.

We can extract two pieces of information from the pipeline:

* Selected features: which features passed through the feature selection step and are actually used by the model.
* Feature importance: the importance scores of these features as calculated by the Random Forest in the modelling step.

In [ ]:
selected_features = (X_train.columns[pipeline_clf['feat_selection']
                                     .get_support()])
print(f"Number of selected features: {len(selected_features)}")
selected_features


In [ ]:
print("Feature importance:")
pipeline_clf['model'].feature_importances_

We create a DataFrame that contains these features' importance and plot it as a bar plot, to show the most important features for the model.

In [ ]:
df_feature_importance = (pd.DataFrame(data={
    'Feature': X_train.columns[pipeline_clf['feat_selection'].get_support()],
    'Importance': pipeline_clf['model'].feature_importances_})
    .sort_values(by='Importance', ascending=False)
)

# re-assign best_features order
best_features = df_feature_importance['Feature'].to_list()

# Most important features statement and plot
print(
    f"* These are the {len(best_features)} most important features in "
    f"descending order. The model was trained on them:\n"
    f"{df_feature_importance['Feature'].to_list()}"
)

df_feature_importance.plot(kind='bar', x='Feature', y='Importance')
plt.show()


From the feature importance analysis, only three features show substantial predictive power for loan default:

* ``person_income``: The most influential feature. Borrowers’ income strongly affects the model’s prediction of default risk, with higher income generally reducing default probability.
* ``loan_int_rate``: Interest rate is another key predictor. Higher interest rates are associated with increased default likelihood.
This also reflects the risk associated with a borrower, because interest rates are tied to loan grade; borrowers with lower/worse loan grades receive higher rates, signaling higher risk.
* ``loan_amnt``: Loan amount also plays a significant role. Larger loans increase default risk.

## Evaluate Pipeline on Train and Test Sets

In [ ]:
# From Churnometer Walkthrough Project
def confusion_matrix_and_report(X, y, pipeline, label_map):

    prediction = pipeline.predict(X)

    print('---  Confusion Matrix  ---')
    print(pd.DataFrame(confusion_matrix(y_true=y, y_pred=prediction),
          columns=[["Prediction " + sub for sub in label_map]],
          index=[["Actual " + sub for sub in label_map]]
          ))
    print("\n")

    print('---  Classification Report  ---')
    print(classification_report(y, prediction, target_names=label_map), "\n")


def clf_performance(X_train, y_train, X_test, y_test, pipeline, label_map):
    print("#### Train Set #### \n")
    confusion_matrix_and_report(X_train, y_train, pipeline, label_map)

    print("#### Test Set ####\n")
    confusion_matrix_and_report(X_test, y_test, pipeline, label_map)

Evaluation: We cross check with the metrics defined in the ML business case
- Recall for default ≥ 0.75 – to minimize false negatives (high-risk borrowers predicted as safe)
- F1 score ≥ 0.60 – ensures a balance between recall and precision

Metrics should be met both on training and test sets.

In [ ]:
clf_performance(X_train=X_train, y_train=y_train,
                X_test=X_test, y_test=y_test,
                pipeline=pipeline_clf,
                label_map= ['No Default', 'Default'] 
                )

The model achieves a recall of 0.82 on the training set and 0.80 on the test set, both above the business threshold of 0.75, ensuring that high-risk borrowers are correctly identified. 

The F1 score is 0.80 on training and 0.61 on testing, showing a drop due to lower precision on unseen data but still above the minimum requirement of 0.60.

Performance is reasonably consistent between training and test sets, and the high recall confirms the model’s effectiveness at capturing defaulters, fulfilling the key business objectives.

# Step 3: Refit pipeline with best features

To prepare the model for deployment as a real-world system, we refit the pipeline using only the three most important features identified during feature importance analysis. This step ensures that the deployed model remains accurate, interpretable, and efficient, focusing only on the variables that truly drive default predictions.

## Refit ML Pipeline and Resampling

In theory, a pipeline fitted using only the most important features should produce similar results to one fitted with all variables and feature selection.

However, in this project we apply SMOTE to balance the training set. This resampling slightly alters the data, so some differences in performance are expected. Nevertheless, we do not expect large deviations, and overall model behavior should remain consistent.

## Rewrite ML pipeline for Data Cleaning and Feature Engineering

In [ ]:
best_features

We create a new pipeline using only the three most important features.

Since we now focus on these key features, SmartCorrelatedSelection is no longer needed, simplifying the pipeline and making it more suitable for deployment.

In [ ]:
def PipelineDataClAndFeatureEng():
    pipeline_base = Pipeline([
        ("MedianImputer", MeanMedianImputer(imputation_method='median', 
                                            variables=['loan_int_rate'])),
        ("Winsorizer", Winsorizer(capping_method='iqr', fold=5, tail='right', 
                                  variables=['person_income', 'loan_amnt', 
                                             'loan_int_rate'])),
        ('log_transform', vt.LogTransformer(variables=['person_income',])),
        ('power_transform', vt.PowerTransformer(variables=['loan_amnt']))
    ])

    return pipeline_base

## Rewrite ML Pipeline for Modelling

Since feature selection is no longer required, SelectFromModel is removed from the modelling pipeline.

In [ ]:
def PipelineClf(model):
    pipeline_base = Pipeline([
        ("scaler", StandardScaler()),
        ("model", model),
    ])

    return pipeline_base

## Split Train Test Set, considering only with best features

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop([target_var], axis=1),
    df[target_var],
    test_size=0.2,
    random_state=0,
)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)


We filter only the most important variables

In [ ]:
X_train = X_train.filter(best_features)
X_test = X_test.filter(best_features)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
X_train.head(3)

## Handle Target Imbalance

In [ ]:
pipeline_data_cl_feat_eng = PipelineDataClAndFeatureEng()
X_train = pipeline_data_cl_feat_eng.fit_transform(X_train)
X_test = pipeline_data_cl_feat_eng.transform(X_test)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

Check Train Set Target distribution

In [ ]:

y_train.value_counts().plot(kind='bar', title='Train Set Target Distribution')
plt.show()

Use SMOTE to balance Train Set target

In [ ]:
oversample = SMOTE(sampling_strategy='minority', random_state=0)
X_train, y_train = oversample.fit_resample(X_train, y_train)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

Check Train Set Target distribution after SMOTE

In [ ]:
y_train.value_counts().plot(kind='bar',title='Train Set Target Distribution')
plt.show()

## Grid Search CV: Sklearn

Next we will conduct another grid search using the most suitable model from the last section and its best hyperparameter configuration.

Model from last grid search

In [ ]:
models_search 

And the best parameters from the last grid search 

In [ ]:
best_parameters

Save best hyperparameter values as a list

In [ ]:
params_search = {'RandomForestClassifier':  {
    'model__n_estimators': [1000],
    'model__min_samples_split': [5],
    'model__min_samples_leaf': [3],
    'model__max_features': ['sqrt'],
    'model__max_depth': [6],
    'model__criterion': ['gini'],
    'model__class_weight': [None]
    }
}
params_search

GridSearch CV

In [ ]:
quick_search = HyperparameterOptimizationSearch(models=models_search, 
                                                params=params_search)
quick_search.fit(X_train, y_train,
           scoring = make_scorer(hybrid_score),
           n_jobs=-1, cv=5)

Check results

The summary performance of the reduced-feature pipeline is 0.79, slightly lower than the previous 0.80.

This difference is expected because SMOTE resampling was applied using only the three selected features, whereas previously all variables were used. The resulting training set is therefore slightly different, leading to a small change in pipeline performance.

Overall, the pipeline remains robust, and we consider this our best ML pipeline for deployment.

In [ ]:
grid_search_summary, grid_search_pipelines = (quick_search
                                              .score_summary(
                                                  sort_by='mean_score'))
grid_search_summary 

Define the best clf pipeline

In [ ]:
best_model = grid_search_summary.iloc[0, 0]
pipeline_clf = grid_search_pipelines[best_model].best_estimator_
pipeline_clf

## Assess feature importance

In [ ]:

# We don't have feature selection,
# so the best features are the columns from the train set.
best_features = X_train.columns

# create DataFrame to display feature importance
df_feature_importance = (pd.DataFrame(data={
    'Feature': best_features,
    'Importance': pipeline_clf['model'].feature_importances_})
    .sort_values(by='Importance', ascending=False)
)

# Most important features statement and plot
print(
    f"* These are the {len(best_features)} most important features in "
    f"descending order. The model was trained on them:\n"
    f"{df_feature_importance['Feature'].to_list()}"
)

df_feature_importance.plot(kind='bar', x='Feature', y='Importance')
plt.show()

* The features have the same relevance as before

## Evaluate Pipeline on Train and Test Sets

Evaluation: We cross check with the metrics defined in the ML business case
- Recall for default ≥ 0.75 – to minimize false negatives (high-risk borrowers predicted as safe)
- F1 score ≥ 0.60 – ensures a balance between recall and precision

Metrics should be met both on training and test sets.

In [ ]:
clf_performance(X_train=X_train, y_train=y_train,
                X_test=X_test, y_test=y_test,
                pipeline=pipeline_clf,
                label_map= ['No Default', 'Default'] 
                )

The model achieves a recall of 0.80 on the training set and 0.78 on the test set, both above the business threshold of 0.75, ensuring that high-risk borrowers are correctly identified. 

The F1 score is 0.80 on training and 0.62 on testing, showing a drop due to lower precision on unseen data but still above the minimum requirement of 0.60.

Performance is reasonably consistent between training and test sets, and the high recall confirms the model’s effectiveness at capturing defaulters, fulfilling the key business objectives.

* Overall the results are very similar to the results we previously had for the pipeline with all features

# Step 4: Push files to Repo

We will generate the following files
* Train set
* Test set
* Data cleaning and Feature Engineering pipeline
* Modeling pipeline
* features importance plot

In [ ]:
# First, we set a version tag, which will be the folder name. 
# It's appropriate since it's a form
# of version control.
version = 'v2'
file_path = f'outputs/ml_pipeline/predict_default/{version}'

# create a folder to store the files.
try:
    os.makedirs(name=file_path)
except Exception as e:
    print(e)

## Train Set

We will save the train and test set since we need them to evaluate the model in the dashboard.

Note that the data cleaning and feature engineering
pipeline have transformed the features. Also SMOTE has already been applied to the train set.

In [ ]:
print(X_train.shape)
X_train.head()

In [ ]:
X_train.to_csv(f"{file_path}/X_train.csv", index=False)

In [ ]:
y_train

In [ ]:
y_train.to_csv(f"{file_path}/y_train.csv", index=False)

## Test Set

In [ ]:
print(X_test.shape)
X_test.head()

In [ ]:
X_test.to_csv(f"{file_path}/X_test.csv", index=False)

In [ ]:
y_test

In [ ]:
y_test.to_csv(f"{file_path}/y_test.csv", index=False)

## ML Pipelines: Data Cleaning and Feat Eng pipeline and Modelling Pipeline

We will save 2 pipelines: 
* Both should be used in conjunction to predict Live Data.
* To predict on Train Set, Test Set we use only pipeline_clf, since the data is already processed.



Pipeline responsible for Data Cleaning and Feature Engineering.


In [ ]:
pipeline_data_cl_feat_eng

In [ ]:
joblib.dump(value=pipeline_data_cl_feat_eng ,
            filename=f"{file_path}/clf_pipeline_data_cl_feat_eng.pkl")

Pipeline responsible for Feature Scaling, and Model

In [ ]:
pipeline_clf

In [ ]:
joblib.dump(value=pipeline_clf ,
            filename=f"{file_path}/clf_pipeline_model.pkl")

## Feature Importance plot

In [ ]:
df_feature_importance.plot(kind='bar',x='Feature',y='Importance')
plt.show()

In [ ]:
df_feature_importance.plot(kind='bar', x='Feature', y='Importance')
plt.savefig(f'{file_path}/features_importance.png', bbox_inches='tight')

---

# Conclusions and Next Steps

We developed a predictive model to estimate the likelihood of borrower default based on key financial and personal features. 
The performance of the model is reasonably consistent between training and test sets, and the high recall confirms the model’s effectiveness at capturing defaulters, fulfilling the key business objectives.

Next Steps:

* As a next step, we will perform a cluster analysis to segment borrowers based on shared characteristics. Combining the prediction model with cluster insights in the dashboard will provide a deeper business understanding, enabling more targeted risk management and tailored strategies for different borrower segments.